# 处理数据
Pytorch有两个处理数据的方法，`torch.utils.data.DataLoader`和`torch.utils.data.Dataset`，`Dataset`存储样本及其相应的标签，`DataLoader`包装一个`Dataset`的迭代器。

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

print(torch.__version__)

Pytorch提供特定领域的库，例如`TorchText`, `TorchVision`和`TorchAudio`，所以这些库都包含数据集。

`torchvision.datasets`模块包含许多现实世界视觉数据`Dataset`对象，例如CIFAR、COCO([支持的数据](https://pytorch.org/vision/stable/datasets.html))，下面使用FashionMNIST数据集。每个TorchVsion Dataset都包含两个参数: `transform`和`target_transform`分别修改样本和标签。

In [ ]:
# 下载训练数据
training_data = datasets.FashionMNIST(
    root="/Users/dsz/Data/Vision/FashionMNIST",
    train=True,
    download=True,
    transform=ToTensor(),
)

In [ ]:
# 下载测试数据
test_data = datasets.FashionMNIST(
    root="/Users/dsz/Data/Vision/FashionMNIST",
    train=False,
    download=True,
    transform=ToTensor(),
)

我们将`Dataset`作为参数传递给`DataLoader`，这在我们的数据集上包装了一个迭代器，并支持自动批处理、采样、洗牌和多进程数据加载。在这里我们定义了一个64的批处理大小，即数据加载器可迭代中的每一个元素将返回一批64个特征和标签

In [ ]:
batch_size = 64

# 创建数据加载器
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# 创建模型
要在PyTorch中定义神经网络，我们需要创建一个继承自`nn.Module`的类，`__init__`函数中定义网络的层，并在`forward`函数中指定数据如何通过网络。为了加速神经网络的运行，我们将其移至GPU或MPS（如果可用）。

In [ ]:
# 选择CPU、GPU或MPS设备进行训练
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 定义模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

# 优化模型参数
为了训练模型，我们需要一个损失函数和一个优化器

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在单个训练循环中，模型对训练数据进行预测(分批喂训练数据集)，并反向传播预测误差以调整模型的参数

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 计算预测误差
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

我们还会根据测试数据集检查模型的性能，以确保它在学习

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程经过多次迭代(epochs)。在每个迭代期间，模型都会学习参数，以做出更好的预测。我们打印模型在每个epoch的准确率和损失；我们希望看到准确率随着每个epoch的增加而增加，损失随着每个epoch的增加而减少。

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

# 保存模型
保存模型的常用方法是序列化内部状态字典（包含模型参数）。

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

# 加载模型
加载模型的过程包括重新创建模型结构并将状态字典加载到模其中

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

该模型可用于进行预测

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.to(device)
model.eval()
x, y = test_data[0][0].to(device), test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

In [ ]:
import matplotlib.pyplot as plt

# 获取第一个样本的图像和标签
image, label = test_data[0]

# 因为image是一个[1, 28, 28]的Tensor，我们需要使用squeeze()方法去掉通道维度
image = image.squeeze()

# 使用matplotlib显示图像
plt.imshow(image, cmap='gray')  # 选择灰度颜色映射
plt.title(f'Label: {label}')  # 显示标签作为标题
plt.axis('off')  # 不显示坐标轴
plt.show()
